In [1]:
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
X = np.ones((10000,1000, 100))
Y = np.concatenate((np.ones(8000), np.zeros(2000)))

In [3]:
X_1, X_2, Y_1, Y_2 = train_test_split(X, Y, test_size=0.2, stratify=Y)

In [6]:
print(X_2.shape)

(2000, 1000, 100)


In [18]:
print(X_1.shape)

(8000, 1000, 100)


In [129]:
print(sum(Y_1 == 0))

1600


In [66]:
from sklearn.model_selection import StratifiedKFold, KFold
import numpy as np
X, y = np.ones((50, 1)), np.hstack(([0] * 45, [1] * 5))
skf = StratifiedKFold(n_splits=3)
for train, test in skf.split(X, y):
    print('train -  {}   |   test -  {}'.format(
        np.bincount(y[train]), np.bincount(y[test])))
    print(y[train])
    
# kf = KFold(n_splits=3)
# for train, test in kf.split(X, y):
#     print('train -  {}   |   test -  {}'.format(
#         np.bincount(y[train]), np.bincount(y[test])))

train -  [30  3]   |   test -  [15  2]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1]
train -  [30  3]   |   test -  [15  2]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1]
train -  [30  4]   |   test -  [15  1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1]


In [130]:
import torch 

In [170]:
a = torch.ones((3,1,1,1))

In [171]:
a.shape

torch.Size([3, 1, 1, 1])

In [172]:
b = torch.linalg.vector_norm(a, ord=2, dim=(1,2,3), keepdim=True)

In [173]:
b.shape

torch.Size([3, 1, 1, 1])

In [174]:
b

tensor([[[[1.]]],


        [[[1.]]],


        [[[1.]]]])

In [1]:
# Option 1: Auto-reload only the specific data module
%load_ext autoreload
%autoreload 1
%aimport data

from data import Cifar10
from torchvision.transforms import v2

In [2]:
train_data = Cifar10(True, v2.ToTensor())

/home/treerspeaking/miniforge3/lib/python3.10/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [ ]:
label_dataloader, unlabeled_dataloader = train_data.get_dataloader(labeled_batch_size=2, labeled_num_worker=4,shuffle = True, unlabeled=True, labeled_size=2000, unlabeled_batch_size=32, unlabeled_num_worker = 2,seed= None)

In [ ]:
import pytorch_lightning as pl
from torch.utils.data import DataLoader

# Custom combined dataloader class
class CombinedDataloader:
    def __init__(self, labeled_dataloader, unlabeled_dataloader):
        self.labeled_dataloader = labeled_dataloader
        self.unlabeled_dataloader = unlabeled_dataloader
        self.labeled_iter = iter(labeled_dataloader)
    
    def __iter__(self):
        for unlabeled_batch in self.unlabeled_dataloader:
            try:
                labeled_batch = next(self.labeled_iter)
            except StopIteration:
                self.labeled_iter = iter(self.labeled_dataloader)
                labeled_batch = next(self.labeled_iter)
            yield labeled_batch, unlabeled_batch
    
    def __len__(self):
        return len(self.unlabeled_dataloader)

# LightningDataModule for semi-supervised learning
class SemiSupervisedDataModule(pl.LightningDataModule):
    def __init__(self, labeled_inputs, labels, unlabeled_inputs, batch_size_labeled=10, batch_size_unlabeled=50):
        super().__init__()
        self.labeled_dataset = LabeledDataset(labeled_inputs, labels)
        self.unlabeled_dataset = UnlabeledDataset(unlabeled_inputs)
        self.batch_size_labeled = batch_size_labeled
        self.batch_size_unlabeled = batch_size_unlabeled
    
    def train_dataloader(self):
        labeled_dataloader = DataLoader(
            self.labeled_dataset,
            batch_size=self.batch_size_labeled,
            shuffle=True
        )
        unlabeled_dataloader = DataLoader(
            self.unlabeled_dataset,
            batch_size=self.batch_size_unlabeled,
            shuffle=True
        )
        return CombinedDataloader(labeled_dataloader, unlabeled_dataloader)

In [5]:
import numpy as np

a = iter(np.arange(10))
b = iter(np.arange(5))

c = zip(a,b)

In [3]:
for i in c:
    print(i)

(np.int64(0), np.int64(0))
(np.int64(1), np.int64(1))
(np.int64(2), np.int64(2))
(np.int64(3), np.int64(3))
(np.int64(4), np.int64(4))


In [8]:
import numpy as np

a = iter(np.arange(10))
b = iter(np.arange(5))

c = zip(a,b)

while True:
    print(next(c))

(np.int64(0), np.int64(0))
(np.int64(1), np.int64(1))
(np.int64(2), np.int64(2))
(np.int64(3), np.int64(3))
(np.int64(4), np.int64(4))


StopIteration: 

In [2]:
class A():
    def __init__(self,):
        self.a = 1
    
    def func():
        print(1)
    

In [3]:
a = A()
print(a.func())

TypeError: A.func() takes 0 positional arguments but 1 was given

In [11]:
from torch import tensor
from torchmetrics import Accuracy

In [18]:
target = tensor([0, 1, 2])
preds = tensor([[0.9, 0.1, 0], [0.3, 0.7, 0.6], [0.2, 0.2, 99]])
accuracy = Accuracy(task="multiclass", num_classes=3)
accuracy(preds, target)

tensor(1.)

In [19]:
preds_2 = tensor([[0.9, 0.1, 0], [0.3, 0.7, 0.6], [0.2, 0.2, 1]])

accuracy(preds_2, target)

tensor(1.)